In [1]:
!pip install pennylane
!pip install numpy
!pip install pandas
!pip install scikit-learn

In [2]:
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from joblib import Parallel, delayed
from functools import lru_cache

# Quantum device setup with PennyLane
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit for embedding classical data into quantum states
@qml.qnode(dev)
def quantum_circuit(inputs):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
        qml.RY(inputs[i], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Cache quantum circuit results for repeated inputs
@lru_cache(maxsize=None)
def cached_quantum_circuit(inputs_tuple):
    inputs = np.array(inputs_tuple)
    return quantum_circuit(inputs)

# Define the quantum kernel with parallelization
def quantum_kernel(X1, X2):
    def kernel_element(x1, x2):
        return np.dot(cached_quantum_circuit(tuple(x1)), cached_quantum_circuit(tuple(x2)))
    
    # Parallel computation
    result = Parallel(n_jobs=-1)(delayed(kernel_element)(x1, x2) for x1 in X1 for x2 in X2)
    return np.array(result).reshape(len(X1), len(X2))

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Normalize the data
X = X / np.linalg.norm(X, axis=1, keepdims=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Classical Machine Learning (SVM)
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)
y_pred_cml = svm.predict(X_test)
accuracy_cml = accuracy_score(y_test, y_pred_cml)
print("Classical Machine Learning Accuracy:", accuracy_cml)

# Quantum Machine Learning (QSVM) using PennyLane
X_train_quantum = quantum_kernel(X_train, X_train)
X_test_quantum = quantum_kernel(X_test, X_train)

qsvm = SVC(kernel='precomputed')
qsvm.fit(X_train_quantum, y_train)
y_pred_qml = qsvm.predict(X_test_quantum)
accuracy_qml = accuracy_score(y_test, y_pred_qml)
print("Quantum Machine Learning Accuracy:", accuracy_qml)

# Compare the accuracy of CML and QML
if accuracy_qml > accuracy_cml:
    print("Quantum Machine Learning outperforms Classical Machine Learning!")
else:
    print("Classical Machine Learning outperforms Quantum Machine Learning!")

# Large dataset experiment
n_samples = 100000
n_features = 10
X_large = np.random.rand(n_samples, n_features)
y_large = np.random.randint(0, 2, n_samples)

X_train_large, X_test_large, y_train_large, y_test_large = train_test_split(X_large, y_large, test_size=0.2, random_state=42)

# Classical Machine Learning (SVM) on large dataset
svm_large = SVC(kernel='linear', C=1)
svm_large.fit(X_train_large, y_train_large)
y_pred_cml_large = svm_large.predict(X_test_large)
accuracy_cml_large = accuracy_score(y_test_large, y_pred_cml_large)
print("Classical Machine Learning Accuracy on Large Dataset:", accuracy_cml_large)

# Quantum Machine Learning (QSVM) on large dataset using PennyLane
X_train_large = X_train_large / np.linalg.norm(X_train_large, axis=1, keepdims=True)
X_test_large = X_test_large / np.linalg.norm(X_test_large, axis=1, keepdims=True)

X_train_large_quantum = quantum_kernel(X_train_large, X_train_large)
X_test_large_quantum = quantum_kernel(X_test_large, X_train_large)

qsvm_large = SVC(kernel='precomputed')
qsvm_large.fit(X_train_large_quantum, y_train_large)
y_pred_qml_large = qsvm_large.predict(X_test_large_quantum)
accuracy_qml_large = accuracy_score(y_test_large, y_pred_qml_large)
print("Quantum Machine Learning Accuracy on Large Dataset:", accuracy_qml_large)

# Compare the accuracy of CML and QML on large dataset
if accuracy_qml_large > accuracy_cml_large:
    print("Quantum Machine Learning outperforms Classical Machine Learning on Large Dataset!")
else:
    print("Classical Machine Learning outperforms Quantum Machine Learning on Large Dataset!")

Classical Machine Learning Accuracy: 0.6333333333333333


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [ ]:

# Problem Statement:
# This code implements Quantum Machine Learning (QML) using PennyLane to analyze and classify data, specifically with the Iris dataset. The goal is to compare the performance of Quantum Support Vector Machines (QSVM) and Classical Support Vector Machines (SVM) using both small and large datasets.
# Objective:
# Iris Dataset: Perform a classification task using both classical and quantum approaches. Evaluate the performance of QSVM and SVM on this small dataset.
# Large Dataset: Extend the experiment by generating a large synthetic dataset and compare the performance of QSVM with classical SVM.
# Performance Comparison: Compare the accuracy of both models and observe whether QML outperforms classical machine learning, especially on large datasets.
# Results: Visualize and analyze the accuracy of both methods on small and large datasets.

import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# Quantum device setup with PennyLane
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit for embedding classical data into quantum states
@qml.qnode(dev)
def quantum_circuit(inputs):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)
        qml.RY(inputs[i], wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Define the quantum kernel
def quantum_kernel(X1, X2):
    result = []
    for x1 in X1:
        row = []
        for x2 in X2:
            row.append(np.dot(quantum_circuit(x1), quantum_circuit(x2)))
        result.append(row)
    return np.array(result)

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Normalize the data
X = X / np.linalg.norm(X, axis=1, keepdims=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Classical Machine Learning (SVM)
svm = SVC(kernel='linear', C=1)
svm.fit(X_train, y_train)
y_pred_cml = svm.predict(X_test)
accuracy_cml = accuracy_score(y_test, y_pred_cml)
print("Classical Machine Learning Accuracy:", accuracy_cml)

# Quantum Machine Learning (QSVM) using PennyLane
X_train_quantum = quantum_kernel(X_train, X_train)
X_test_quantum = quantum_kernel(X_test, X_train)

qsvm = SVC(kernel='precomputed')
qsvm.fit(X_train_quantum, y_train)
y_pred_qml = qsvm.predict(X_test_quantum)
accuracy_qml = accuracy_score(y_test, y_pred_qml)
print("Quantum Machine Learning Accuracy:", accuracy_qml)

# Compare the accuracy of CML and QML
if accuracy_qml > accuracy_cml:
    print("Quantum Machine Learning outperforms Classical Machine Learning!")
else:
    print("Classical Machine Learning outperforms Quantum Machine Learning!")

# Large dataset experiment
n_samples = 100000
n_features = 10
X_large = np.random.rand(n_samples, n_features)
y_large = np.random.randint(0, 2, n_samples)

X_train_large, X_test_large, y_train_large, y_test_large = train_test_split(X_large, y_large, test_size=0.2, random_state=42)

# Classical Machine Learning (SVM) on large dataset
svm_large = SVC(kernel='linear', C=1)
svm_large.fit(X_train_large, y_train_large)
y_pred_cml_large = svm_large.predict(X_test_large)
accuracy_cml_large = accuracy_score(y_test_large, y_pred_cml_large)
print("Classical Machine Learning Accuracy on Large Dataset:", accuracy_cml_large)

# Quantum Machine Learning (QSVM) on large dataset using PennyLane
X_train_large = X_train_large / np.linalg.norm(X_train_large, axis=1, keepdims=True)
X_test_large = X_test_large / np.linalg.norm(X_test_large, axis=1, keepdims=True)

X_train_large_quantum = quantum_kernel(X_train_large, X_train_large)
X_test_large_quantum = quantum_kernel(X_test_large, X_train_large)

qsvm_large = SVC(kernel='precomputed')
qsvm_large.fit(X_train_large_quantum, y_train_large)
y_pred_qml_large = qsvm_large.predict(X_test_large_quantum)
accuracy_qml_large = accuracy_score(y_test_large, y_pred_qml_large)
print("Quantum Machine Learning Accuracy on Large Dataset:", accuracy_qml_large)

# Compare the accuracy of CML and QML on large dataset
if accuracy_qml_large > accuracy_cml_large:
    print("Quantum Machine Learning outperforms Classical Machine Learning on Large Dataset!")
else:
    print("Classical Machine Learning outperforms Quantum Machine Learning on Large Dataset!")
